# *``lca_graphic``* : **A toolbox for daily LCA users**

---

**Purpose = to give understandable graphs for the daily LCA users to quickly analyze his LCA**
<br>
---

**Author** : Teo Lavisse, teo.lavisse@cea.fr       


## Initializing

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json
import os
import matrix_utils as mu
import bw_processing as bp
import math
import warnings
import ipywidgets as widgets
from ipywidgets import interact
import bw2analyzer as ba
import seaborn as sns
import textwrap

from utils import *
from dashboards import *

In [2]:
bd.projects.set_current("Visualization")

### Importing Databases

In [3]:
bd.databases


Databases dictionary with 3 object(s):
	US EEIO 1.1
	biosphere3
	ecoinvent 3.9_cutoff

In [4]:
db_USLCI=bd.Database("US EEIO 1.1")
db_EI=bd.Database("ecoinvent 3.9_cutoff")

###  Get activities and methods

#### US LCI

Get 2 activities and methods from the database

In [5]:
window_metal=next(node for node in db_USLCI if node['type'] == 'product' if node['name']=="Metal windows, doors, and architectural products; at manufacturer")
window_metal

'Metal windows, doors, and architectural products; at manufacturer' (, United States, ('31-33: Manufacturing', '3323: Architectural and Structural Metals Manufacturing'))

In [6]:
window_wood=next(node for node in db_USLCI if node['type'] == 'product' if node['name']=="Wooden windows, door, and flooring; at manufacturer")
window_wood

'Wooden windows, door, and flooring; at manufacturer' (, United States, ('31-33: Manufacturing', '3219: Other Wood Product Manufacturing'))

In [7]:
fu={window_metal:1,window_wood:1}

In [8]:
methods = [
    ('Impact Potential', 'GCC'),
    ('Impact Potential', 'HTOX'),
    ('Resource Use', 'LAND'),
    ('Resource Use', 'WATR'),
]

#### EI

In [9]:
natural_gas = [act for act in db_EI if "market for natural gas, high pressure" in act['name'] if 'CH' in act['location'] if 'cubic meter' in act['unit']][0]

In [10]:
biomethane = [act for act in db_EI if "market for biomethane, high pressure" in act['name'] if 'CH' in act['location'] if 'cubic meter' in act['unit']][0]

In [11]:
fu = {natural_gas: 1, biomethane: 1}

In [12]:
CC = [method for method in bd.methods if "IPCC 2021" in str(method) and "GWP100" in str(method)][0]
ADP = [method for method in bd.methods if "ADP" in str(method) if "ultimate reserves" in str(method) if 'EF v3.1 no LT' in str(method)][0]
TOX = [method for method in bd.methods if "human toxicity: carcinogenic no LT" in str(method) if 'EF v3.1' in str(method)][0]
LU = [method for method in bd.methods if 'land use no LT' in str(method) if 'soil quality index' in str(method) if 'EF v3.1' in str(method)][0]

impact_categories = [CC, ADP, TOX, LU]

In [13]:
impact_categories

[('IPCC 2021 no LT',
  'climate change no LT',
  'global warming potential (GWP100) no LT'),
 ('EF v3.1 no LT',
  'material resources: metals/minerals no LT',
  'abiotic depletion potential (ADP): elements (ultimate reserves) no LT'),
 ('EF v3.1 no LT',
  'human toxicity: carcinogenic no LT',
  'comparative toxic unit for human (CTUh) no LT'),
 ('EF v3.1 no LT', 'land use no LT', 'soil quality index no LT')]

In [14]:
fu

{'market for natural gas, high pressure' (cubic meter, CH, None): 1,
 'market for biomethane, high pressure' (cubic meter, CH, None): 1}

##  Perform basic lca computations

In [15]:
fu

{'market for natural gas, high pressure' (cubic meter, CH, None): 1,
 'market for biomethane, high pressure' (cubic meter, CH, None): 1}

In [16]:
df=lca_comparison(fu, impact_categories)
df

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)","(EF v3.1 no LT, material resources: metals/minerals no LT, abiotic depletion potential (ADP): elements (ultimate reserves) no LT)","(EF v3.1 no LT, human toxicity: carcinogenic no LT, comparative toxic unit for human (CTUh) no LT)","(EF v3.1 no LT, land use no LT, soil quality index no LT)"
"market for natural gas, high pressure",0.640412,6.740114e-07,3.084988e-10,0.526620
"market for biomethane, high pressure",0.281268,2.397764e-06,1.322233e-10,3.199694


In [17]:
df_norm=df.T.apply(lambda x: x/x.max(), axis=1)*100 #to normalize the results for each impact category
df_norm

,"market for natural gas, high pressure","market for biomethane, high pressure"
"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)",100.000000,43.919753
"(EF v3.1 no LT, material resources: metals/minerals no LT, abiotic depletion potential (ADP): elements (ultimate reserves) no LT)",28.110001,100.000000
"(EF v3.1 no LT, human toxicity: carcinogenic no LT, comparative toxic unit for human (CTUh) no LT)",100.000000,42.860234
"(EF v3.1 no LT, land use no LT, soil quality index no LT)",16.458453,100.000000


## Perform a basic contribution analysis

In [18]:
act_topscore(fu, CC)

'market for natural gas, high pressure' (cubic meter, CH, None)

In [19]:
df=contributions_df(biomethane,CC,limit=5,limit_type='number',norm=True, group_by_other=True)
df

,"(IPCC 2021 no LT, climate change no LT, global warming potential (GWP100) no LT)"
5bd4d1d3ea2ad8b6f9288db105968f92,3.524154
c82d278ad302ca45c2345063cdce422d,5.797948
e413022065fd99c773b8664679178c5f,11.028949
91c36ddeefe08ea0d677c17707cd15ab,12.690830
Others,29.393422
45dcb00f3f53f203d47028257b0d747d,37.564699


## Display all thes informations in a nice dashboard

### Defining colors

In [20]:
#define standard color palette:
colors = ["#F08C2E", "#7f6000", "#72AF42", "#A32683"]

#create longer color list for complex figures
color_div_YlBr=sns.color_palette('YlOrBr',6)
color_seq_green=sns.color_palette('Greens',6)
color_seq_RdPu=sns.color_palette('RdPu',6)
color_seq_org=sns.color_palette('Oranges',5)

colors.extend(colors)
# colors.extend(color_div_YlBr)
# colors.extend(color_seq_green)
# colors.extend(color_seq_RdPu)
# colors.extend(color_seq_org)

df_color=pd.DataFrame(index=impact_categories,data=[colors[c] for c in range(len(impact_categories))]).T

### Dashboard compare

In [21]:
compare(fu, impact_categories, sharex=False, cols=2, func_unit="cubic meter")

### Dashboard hotspots

In [22]:
hotspots(fu, impact_categories, limit=0.05)

### Dashboard impact_transfer

In [23]:
impact_transfer(fu, impact_categories, reference_category=CC, limit=5, cols=2, func_unit="cubic meter")

## Generic function : lca_graphic

In [27]:
lca_graphic(fu, impact_categories)